In [1]:
import pandas as pd
import numpy as np
from _dictionary import sts_parameters, predict_parameters

In [4]:
path_DB = 'C:\\Users\\lucas.oliveira\\OneDrive - ARCADIS\\Desktop\\Documentos Lucas\\Pos Graduação Lucas\\TCC\\documentos\\DB_FUT\\'

In [5]:
dt_info = pd.read_excel(path_DB + 'dt_info.xlsx')
dt_partida = pd.read_excel(path_DB + 'dt_partida.xlsx')
dt_temporada = pd.read_excel(path_DB + 'dt_temporada.xlsx', dtype = {'Equipe_ID':np.int32})
dt_time_sts = pd.read_excel(path_DB + 'dt_time_sts.xlsx')
dt_time = pd.read_excel(path_DB + 'dt_time.xlsx')


In [6]:
dt_time_sts

,Equipe_ID,Partida_ID,GolsFeitos,GolsSofridos,Faltas,Escanteios,Cruzamentos,Contatos,Botedefensivo,Cortes,QtdDefesas,Impedimentos,Tirodemeta,Bolaslongas,posse,acertodepasses,chutesaogol,defesasPorcentagem,Jogadasaéreas,Cobrançadelateral
0,25,1,1,1,7,4,21,537,19,24,18,3,10,57,42.0,77.0,27.0,25.0,0,0
1,19,2,1,1,10,1,15,810,19,9,12,1,10,54,67.0,84.0,40.0,0.0,0,0
2,26,3,3,2,15,5,27,483,12,11,28,2,10,54,38.0,86.0,56.0,71.0,0,0
3,38,4,4,0,19,3,18,590,16,16,18,5,11,47,51.0,86.0,56.0,100.0,0,0
4,7,5,6,2,11,14,30,487,22,7,16,4,4,44,40.0,81.0,37.0,60.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6065,50,3031,1,0,18,4,12,544,14,11,12,2,13,59,49.0,88.0,36.0,100.0,0,0
6066,32,3032,1,0,17,6,20,452,15,21,18,6,8,58,36.0,75.0,45.0,100.0,0,0
6067,22,3033,0,0,9,1,7,668,19,7,10,2,9,76,51.0,83.0,17.0,100.0,0,0
6068,15,3034,1,1,16,12,40,545,9,10,20,3,16,79,52.0,76.0,45.0,83.0,0,0


In [7]:
def calculate_winner(dt_partida):
    dt_partida['Diferenca'] = dt_partida['m_placar'] - dt_partida['v_placar']

    dt_partida.loc[dt_partida['Diferenca'] == 0, 'winner'] = 2

    dt_partida.loc[dt_partida['Diferenca'] > 0, 'winner'] = 1

    dt_partida.loc[dt_partida['Diferenca'] < 0, 'winner'] = 3

    dt_partida = dt_partida.drop(['Diferenca'], axis = 'columns')

    return dt_partida

In [8]:
def calculate_score(dt_time_sts):
    dt_time_sts['Diferenca'] = dt_time_sts['GolsFeitos'] - dt_time_sts['GolsSofridos']

    dt_time_sts.loc[dt_time_sts['Diferenca'] == 0, 'score'] = 1

    dt_time_sts.loc[dt_time_sts['Diferenca'] > 0, 'score'] = 3

    dt_time_sts.loc[dt_time_sts['Diferenca'] < 0, 'score'] = 0

    dt_time_sts = dt_time_sts.drop(['Diferenca'], axis = 'columns')

    return dt_time_sts

In [9]:
def calculate_last_score(dt_time_sts,dt_time, dt_info,parameters,mdmv):
    dt_info = dt_info[['Partida_ID', 'Data']]
    dt_time_sts = pd.merge(dt_time_sts, dt_info, on= 'Partida_ID', how='inner')
    dt_time_sts['Data'] = pd.to_datetime(dt_time_sts['Data'],)

    df_sts = pd.DataFrame()
    for idx in dt_time.itertuples():
        try:
            nColumns = []
            df = dt_time_sts[dt_time_sts['Equipe_ID'] == idx.Equipe_ID ]
            df = df.sort_values(by=['Data'])
            for param in parameters:
                nColum = param+'_mdmv_'+str(mdmv)
                nColumns.append(nColum)
                df[nColum] = df[param].rolling(mdmv).sum()
                df = df.reset_index(drop=True)
                
                df[nColum] = df[nColum].fillna(df.loc[mdmv-1,nColum])
            #cols = ['Equipe_ID','Partida_ID','Data']
            #cols.extend(nColumns)
            #df = df[cols]
            df_sts = df_sts.append(df)
            #print(df)
        except:
            print(idx.Equipe_ID)
    df_sts = df_sts.drop(['Data'], axis = 'columns')
    df_sts = df_sts.reset_index(drop=True)
    return df_sts

In [10]:
def calculate_mdmv(mdmv, parameters,dt_time_sts,dt_info,dt_time):
    dt_info = dt_info[['Partida_ID', 'Data']]
    dt_time_sts = pd.merge(dt_time_sts, dt_info, on= 'Partida_ID', how='inner')
    dt_time_sts['Data'] = pd.to_datetime(dt_time_sts['Data'],)

    df_sts = pd.DataFrame()
    for idx in dt_time.itertuples():
        try:
            nColumns = ['Data']
            df = dt_time_sts[dt_time_sts['Equipe_ID'] == idx.Equipe_ID ]
            df = df.sort_values(by=['Data'])
            for param in parameters:
                nColum = param+'_mdmv_'+str(mdmv)
                nColumns.append(nColum)
                df[nColum] = df[param].rolling(mdmv).mean()
                df = df.reset_index(drop=True)
                
                df[nColum] = df[nColum].fillna(df.loc[mdmv-1,nColum])
            #cols = ['Equipe_ID','Partida_ID','Data']
            #cols.extend(nColumns)
            #df = df[cols]
            df_sts = df_sts.append(df)
            #print(df)
        except:
            print(idx.Equipe_ID)
    df_sts = df_sts.drop(['Data'], axis = 'columns')
    df_sts = df_sts.reset_index(drop=True)
    return df_sts

In [11]:
def merge_time(df_partida,df_time,sufix):
    
    
    df_partida['Partida_Equipe_ID'] = df_partida[sufix+'LastPartida_ID'].astype(str) + "_" + df_partida[sufix+'Equipe_ID'].astype(str)
    df_time['Partida_Equipe_ID'] = df_time['Partida_ID'].astype(str) + "_" + df_time['Equipe_ID'].astype(str)
    df_time = df_time.drop(['Partida_ID','Equipe_ID'], axis = 'columns')
    dict_columns = {}
    for colum in df_time.columns:
        if colum not in ['Partida_Equipe_ID']:
            dict_columns[colum] = sufix+colum 
    
    df_partida = pd.merge(df_partida,df_time, how ='inner', on = 'Partida_Equipe_ID')
    df_partida = df_partida.rename(columns = dict_columns)
    df_partida = df_partida.drop(['Partida_Equipe_ID'], axis = 'columns')
    #print(df_partida.columns)
    return df_partida

In [12]:
def merge_time_atual(df_partida,df_time,sufix,Param):
    
    
    df_partida['Partida_Equipe_ID'] = df_partida['Partida_ID'].astype(str) + "_" + df_partida[sufix+'Equipe_ID'].astype(str)
    df_time['Partida_Equipe_ID'] = df_time['Partida_ID'].astype(str) + "_" + df_time['Equipe_ID'].astype(str)
    df_time = df_time.drop(['Partida_ID','Equipe_ID'], axis = 'columns')
    Param.append('Partida_Equipe_ID')
    print(Param)
    #print(df_time.columns)
    df_time = df_time[Param]
    Param.remove('Partida_Equipe_ID')
    dict_columns = {}
    for colum in df_time.columns:
        if colum not in ['Partida_Equipe_ID','Data']:
            dict_columns[colum] = sufix+'Atual'+colum 
    
    df_partida = pd.merge(df_partida,df_time, how ='left', on = 'Partida_Equipe_ID')
    df_partida = df_partida.rename(columns = dict_columns)
    #df_partida = df_partida.drop(['Data'], axis = 'columns')
    #print(df_partida.columns)
    return df_partida

In [13]:
def merge_aux(df_partida,df_temporada,sufix):
    
    if sufix == 'm_':
        type_info = '1'
    elif sufix == 'v_':
        type_info = '2'
    else:
        type_info = ''

    list_col = ['QtdJogadores', 'MdIdade', 'MdPosse', 'QtdJogosDisputados',
       'QtdGols', 'QtdAssistencias', 'QtdGolsNormais', 'QtdGolPenaltis',
       'QtdPenaltisBatidos', 'QtdCartoesAmarelos', 'QtdCartoesVermelhos',
       'MdGolsPorPartida', 'MdAssistenciasPorPartida',
       'MdGolsAssistenciasPorPartida', 'MdGolsNormaisPorPartida',
       'MdGolsNormaisAssistenciasPorPartida','key']

    df_temporada['key'] = df_temporada['Equipe_ID'].astype(str)+"_"+df_temporada['ano'].astype(str)+"_"+df_temporada['info_type'].astype(str)
    df_temporada = df_temporada[list_col]
    df_partida['ano_anterior'] = df_partida['ano_ref'] - 1
    df_partida[sufix+'key'] = df_partida[sufix+'Equipe_ID'].astype(str)+"_"+df_partida['ano_anterior'].astype(str)+"_"+type_info
    nCol = sufix+'key'
    df_partida = pd.merge(df_partida,df_temporada,how ='inner',left_on=nCol, right_on='key' )
    
    for col in list_col:
    
        df_partida = df_partida.rename(columns = {col:sufix+col})
    df_partida = df_partida.drop([nCol], axis = 'columns')
    return df_partida

In [14]:
def merge_ID(df_partida, df_LastPartida, sufix):
    df_partida['key'] = df_partida['Partida_ID'].astype(str) + "_" + df_partida[sufix+'Equipe_ID'].astype(str)
    df_partida = pd.merge(df_partida,df_LastPartida, on ='key', how = 'inner')
    nCol = sufix + 'LastPartida_ID'
    df_partida = df_partida.rename(columns = {'LastPartida_ID':nCol})
    df_partida = df_partida.drop(['key'], axis = 'columns')
    return df_partida

In [15]:
def get_last_partidaID(df_partida, df_info):
    df_partida = df_partida.sort_values(by = ['Partida_ID'])
    df_partida = df_partida.reset_index(drop=True)
    df_info = df_info[['Partida_ID','Data']]
    df_partida = df_partida[['Partida_ID','m_Equipe_ID','v_Equipe_ID']]
    df_partida = pd.merge(df_partida,df_info, on='Partida_ID',how='inner')

    df = df_partida[['Partida_ID','m_Equipe_ID','Data']].rename(columns = {'m_Equipe_ID':'Equipe_ID'})
    df = df.append(df_partida[['Partida_ID','v_Equipe_ID','Data']].rename(columns = {'v_Equipe_ID':'Equipe_ID'}))
    #df = df.sort_values(by = ['Partida_ID'])
    print(len(df))
    df_PartidaID = pd.DataFrame()
    for ID in list(set(df.Equipe_ID.tolist())):
        df_Equipe = df.loc[df.Equipe_ID == ID,]
        df_Equipe = df_Equipe.sort_values(by = ['Data'])
        df_Equipe['LastPartida_ID'] = df_Equipe.Partida_ID.shift(periods=1)

        df_PartidaID = df_PartidaID.append(df_Equipe)
        #print(df_Equipe)
    df_PartidaID['key'] = df_PartidaID['Partida_ID'].astype(str) + "_" + df_PartidaID['Equipe_ID'].astype(str)
    df_PartidaID = df_PartidaID[['key','LastPartida_ID']]
    df_PartidaID['LastPartida_ID'] = df_PartidaID['LastPartida_ID'].astype(str).str.replace('.0','',regex = False)
    return df_PartidaID

In [16]:
df_LastPartida = get_last_partidaID(dt_partida,dt_info)

6070


In [17]:
df_LastPartida

,key,LastPartida_ID
488,489_1,nan
556,557_1,489
417,418_1,557
705,706_1,418
491,492_1,706
...,...,...
3011,3012_50,3004
3024,3025_50,3012
3030,3031_50,3025
2972,2973_50,3031


In [18]:
df_sts = calculate_mdmv(10,['GolsFeitos','GolsSofridos','Faltas', 'Escanteios', 'Cruzamentos','QtdDefesas', 'Impedimentos','posse', 'chutesaogol'],dt_time_sts,dt_info,dt_time)


51
52


In [19]:
df_sts = calculate_score(df_sts)
df_sts = calculate_last_score(df_sts,dt_time,dt_info,['score'],10)

51
52


In [20]:
dt_partida = calculate_winner(dt_partida)
dt_partida = merge_ID(dt_partida, df_LastPartida,'m_')
dt_partida = merge_ID(dt_partida,df_LastPartida,'v_')

In [21]:
dt_partida = merge_time(dt_partida, df_sts ,"m_")
dt_partida = merge_time(dt_partida, df_sts ,"v_")
len(dt_partida)

2999

In [22]:
predict_parameters = ['winner']

In [23]:
dt_partida = merge_aux(dt_partida,dt_temporada,'m_')
dt_partida = merge_aux(dt_partida,dt_temporada,'v_')

In [24]:
dt_partida = dt_partida.dropna()
len(dt_partida)

2302

In [25]:
#dt_partida.to_excel(path_DB+'DadosTreinamento.xlsx', index = False)

In [28]:
df= dt_partida[[
'winner',
'm_GolsFeitos_mdmv_10',
'm_GolsSofridos_mdmv_10',
'm_Faltas_mdmv_10',
'm_Escanteios_mdmv_10',
'm_Cruzamentos_mdmv_10',
'm_QtdDefesas_mdmv_10',
'm_Impedimentos_mdmv_10',
'm_posse_mdmv_10',
'm_chutesaogol_mdmv_10',
'm_score_mdmv_10',
'v_GolsFeitos_mdmv_10',
'v_GolsSofridos_mdmv_10',
'v_Faltas_mdmv_10',
'v_Escanteios_mdmv_10',
'v_Cruzamentos_mdmv_10',
'v_QtdDefesas_mdmv_10',
'v_Impedimentos_mdmv_10',
'v_posse_mdmv_10',
'v_chutesaogol_mdmv_10',
'v_score_mdmv_10',
'v_MdPosse',
'v_QtdGols',
'v_QtdAssistencias',
'v_MdGolsPorPartida',
'v_MdAssistenciasPorPartida',
'm_MdPosse',
'm_QtdGols',
'm_QtdAssistencias',
'm_MdGolsPorPartida',
'm_MdAssistenciasPorPartida'
]]

In [39]:
len(df[df.m_MdPosse == 0])
df.loc[df.m_MdPosse == 0,'m_MdPosse'] = df.loc[df.m_MdPosse != 0,'m_MdPosse'].mean()
print(
    df.loc[df.m_MdPosse == 0,'m_MdPosse']
)

Series([], Name: m_MdPosse, dtype: float64)


In [27]:
#df.to_excel(path_DB+'DadosTreinamento_cols.xlsx', index = False)